# Ноутбук для решения задач из 4 модуля

In [2]:
!pip install openai langchain tiktoken langchain-openai langchain-community sentence_transformers faiss-cpu pypdf -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from typing import List

## Если используете ключ от OpenAI, запустите эту ячейку 👇







In [ ]:
from langchain_openai import ChatOpenAI
import os
from getpass import getpass


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [4]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2025-05-20 21:28:40--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11316 (11K) [text/plain]
Saving to: ‘utils.py.8’

utils.py.8          100%[===================>]  11.05K  --.-KB/s    in 0.02s   

2025-05-20 21:28:41 (579 KB/s) - ‘utils.py.8’ saved [11316/11316]



In [5]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

## Задание 4.1.5 Кастомный Text Splitter

---



# *Напишите функцию*

In [ ]:
def get_content_from_sections(text: str) -> List[str]:
    # your code here
    return

#Тексты из тестов


In [ ]:

text1 = """
1. Вступление
Описание курса, начальные требования, для кого этот курс.
1.1 Как правильно входить в курс
Цели курса, канал и комьюнити, глоссарий, PET-проект, хард режим vs лайт режим?
1.2 Общий подход и точки улучшения приложений с LLM
Рассказываем почему важно разбираться в LLM. Нужен ли ИИ обычному человеку?
1.3 API ключ курса или от OpenAI?
1.3.1 Ключ от команды курса
Получаем ключ в боте.
1.3.2 Ключ от OpenAI
Получаем официальный ключ от OpenAI.
1.3.3 Ключ от HuggingFace
Ныряем в Open Source и получаем ключ от HuggingFace.
2. Промптинг - объясни LLM, что тебе от неё надо!
2.1 Введение в Prompt Engineering
Поясняем за промпты. Техники и лайфхаки для промптинга. Из чего состоит промпт?
2.2 Дизайн промптов в LangChain
Few-shot learning. Output Parser.
2.2.1 Введение в LangChain
Рассказываем про преимущества LangChain.
3. LangChain или причем тут попугаи?
3.1 Память в LangChain
Переводим LLM в чат-режим. Типы памяти.
3.2 Chains - собери свою цепь
Chains & LCEL.
3.3 Агенты intro
Агенты и цепи. Инструменты (tools).
"""

text2 = """


"""

text3 = """
1. Главный заголовок
Тут содержимое, которое нужно достать из главного заголовка.
1.1 Подзаголовок
Тут содержимое, которое нужно достать из подзаголовка.
"""

text4 = """
1. Первый заголовок

2. Второй заголовок
"""

text5 = """
1. Введение
Краткое описание целей и задач документа.

2. Основные разделы
Краткий обзор основных разделов.

2.1 Первый подраздел
Какие-то мелкие детали.

3. Заключение
Выводы и заключительные комментарии.
4. Приложения
Дополнительная информация и материалы.
"""

# Задание 4.11.1 📚 Литературный RAG из pdf 🤖

Загружаем файлы

In [7]:
!wget https://stepik.org/media/attachments/lesson/1084288/The_Daughter_of_The_Commandant.pdf

--2025-05-20 21:29:02--  https://stepik.org/media/attachments/lesson/1084288/The_Daughter_of_The_Commandant.pdf
Resolving stepik.org (stepik.org)... 178.248.239.111
Connecting to stepik.org (stepik.org)|178.248.239.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198595 (194K) [application/pdf]
Saving to: ‘The_Daughter_of_The_Commandant.pdf.1’

The_Daughter_of_The 100%[===================>] 193.94K   298KB/s    in 0.7s    

2025-05-20 21:29:05 (298 KB/s) - ‘The_Daughter_of_The_Commandant.pdf.1’ saved [198595/198595]



In [6]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1084288/pushkin_questions.csv')

Соберем все компоненты для RAG


<center> <img src='https://github.com/a-milenkin/LLM_practical_course/blob/main/images/RAG.png?raw=1' width="800" height="250">


In [8]:
from langchain.document_loaders import PyPDFLoader


In [9]:
# Определите Document loader для pdf
loader = PyPDFLoader("The_Daughter_of_The_Commandant.pdf")

documents = loader.load()

In [10]:
documents

[Document(metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2009-12-24T14:33:47+03:00', 'author': '<CCF3F1E0F2EEE2E0>', 'moddate': '2009-12-24T14:33:47+03:00', 'title': '<4D6963726F736F667420576F7264202D20CAC45FE8E7EB5FD0EEE4E8ED>', 'source': 'The_Daughter_of_The_Commandant.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='www.a4format.ru  \nВсе произведения школьной программы в кратком изложении. — М.: АСТ, 1996.  \n \nИ.О. Родин, Т.М. Пименова \nКапитанская дочка \nКраткое содержание \nГлава 1 \nСержант гвардии \nГлава открывается биографией Петра Гринева: отец служил, вышел в отставку, \nв семье было девять детей, но все, кроме Петра, умерли в младенчестве. Еще до его появ-\nления на свет Гринева записали в Семеновский полк. До совершеннолетия он считался в \nотпуске. Воспитывает мальчика дядька Савельич, под руководством которого Петруша \nосваивает русскую грамоту и учится судить о достоинствах б

Посмотрите, что хранится в переменной documents - может быть сплитер вам не нужен.

In [11]:
# Определите подходящий Splitter
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # размер каждого фрагмента в символах
    chunk_overlap=200,  # перекрытие между фрагментами для сохранения контекста
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # приоритет разделителей
)

split_documents = splitter.split_documents(documents)  # не create_documents(), а split_documents()
len(split_documents)

14

# Выбираем Embedding models

## 🔱 Эмбеддинги от OpenAI (API)

In [ ]:
# # Если используете ключ от OpenAI запустите эту ячейку
# from langchain_openai import OpenAIEmbeddings

# embeddings_api_model = OpenAIEmbeddings()

In [12]:
# Если используете ключ курса, запустите эту ячейку
from utils import OpenAIEmbeddings

embeddings_api_model = OpenAIEmbeddings(course_api_key=course_api_key)

## 🔱 Эмбеддинги от HuggingFace 🤗
Переключитесь на среду с GPU, если хотите использовать видеокарту.

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

/tmp/ipykernel_51265/3050949720.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

# Выбираем 🗂 Vector Store

In [15]:
from langchain.vectorstores import FAISS

# Выбираем, какую модель эмбеддингов использовать: OpenAI API или HuggingFace
# Для примера берем модель API, но можно заменить на hf_embeddings_model
db = FAISS.from_documents(split_documents, embeddings_api_model)

# Определим 🎣 Retriever

In [16]:
retriever = db.as_retriever(search_kwargs={"k": 3})

# 🚰 RAG Pipeline - подключаем RAG к LLM

In [17]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон
template = """
Answer the question based only on the following context:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"question": RunnablePassthrough(), "context": retriever | format_docs}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
answers = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['question']):
    answer = chain.invoke(text_input)
    answers.append(answer) # Добавляем ответ в список
    #break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 15/15 [00:38<00:00,  2.53s/it]


In [21]:
df['answer'] = answers # Создаём новый столбец из ответов модели

In [22]:
df.to_csv('4_1_11_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)